In [2]:
lr = .015
for i in range(500):
    if (i+1)%15==0:
        lr /= 1.05
    print(lr)

0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.015
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.014285714285714285
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.013605442176870748
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.01295756397797214
0.0129

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import pairwise_distances
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import os
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [2]:
import nltk
import string

def lower(word):
    return word.lower()

lemm = nltk.stem.WordNetLemmatizer()
def lemmatize(word):
    return lemm.lemmatize(word)

punct = set(string.punctuation)
def no_punctuation(word):
    return ''.join(c for c in word if c not in punct)

stop_words = set(nltk.corpus.stopwords.words("english"))
def no_stop_words(word):
    return word if word not in stop_words else ''

strategy_map = {'lo':lower,'lem':lemmatize,
                'punct':no_punctuation,'stop':no_stop_words}

def preprocess(docs,strategies):
    for strategy in strategies:
        new_docs = []
        for doc in docs:
            new_doc = []
            for word in doc:
                transformed = strategy_map[strategy](word)
                if transformed:
                    new_doc.append(transformed)
            new_docs.append(new_doc)
        docs = new_docs
    return docs

In [3]:
documents_train = []
for i in range(10000):
    with open('../descriptions_train/%d.txt' % (i,)) as f:
        documents_train.append(f.read().split())
documents_train = preprocess(documents_train,['lo','punct'])
documents_train_new = []
for i in range(10000):
    documents_train_new.append(TaggedDocument(documents_train[i],[i]))
documents_train = documents_train_new

In [4]:
model = Doc2Vec(documents_train, vector_size=100, window=10, 
                min_count=2, workers=4)

In [5]:
model.train(documents_train,total_examples=model.corpus_count,epochs=100)

In [6]:
documents_train[44]

TaggedDocument(words=['two', 'giraffes', 'in', 'a', 'room', 'with', 'people', 'looking', 'at', 'them', 'two', 'giraffe', 'standing', 'next', 'to', 'each', 'other', 'in', 'a', 'room', 'the', 'giraffe', 'is', 'being', 'kept', 'by', 'itself', 'indoors', 'a', 'man', 'and', 'woman', 'staring', 'at', 'two', 'giraffes', 'through', 'a', 'window', 'a', 'giraffe', 'in', 'a', 'enclosed', 'area', 'is', 'watched', 'by', 'some', 'people'], tags=[44])

In [7]:
model.wv.most_similar("dog",topn=12)

[('cat', 0.5779536962509155),
 ('boy', 0.5342348217964172),
 ('man', 0.523769736289978),
 ('dogs', 0.5213435888290405),
 ('person', 0.5145947933197021),
 ('sandwich', 0.5038636922836304),
 ('hotdog', 0.4963188171386719),
 ('child', 0.47972357273101807),
 ('puppy', 0.4773727059364319),
 ('crowd', 0.45716097950935364),
 ('horse', 0.45696496963500977),
 ('sheep', 0.4551716446876526)]

In [8]:
text_train = np.array([model[i] for i in range(10000)])

In [9]:
text_test = np.zeros((2000,text_train.shape[1]))
documents_test = []
for i in range(2000):
    with open('../descriptions_test/%d.txt' % (i,)) as f:
        documents_test.append(f.read().split())
documents_test = preprocess(documents_test,['lo','punct'])
for i in range(2000):
    text_test[i] = model.infer_vector(documents_test[i])

In [10]:
text_train = np.hstack([text_train,np.ones((10000,1))])
text_test = np.hstack([text_test,np.ones((2000,1))])

In [11]:
text_train.shape,text_test.shape

((10000, 101), (2000, 101))

In [12]:
feats_train = pd.read_csv('../features_train/features_resnet1000intermediate_train.csv',header=None)
feats_train_b = pd.read_csv('../features_train/features_resnet1000_train.csv',header=None)
feats_test = pd.read_csv('../features_test/features_resnet1000intermediate_test.csv',header=None)
feats_test_b = pd.read_csv('../features_test/features_resnet1000_test.csv',header=None)

In [13]:
feats_train_b.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,images_train/5373.jpg,-0.899450,-0.930470,-2.503365,-3.172499,-2.819133,0.992159,-3.698863,0.619991,0.956148,...,-3.021916,2.214253,-1.382491,1.672911,1.014233,2.599949,2.773284,-2.066632,0.385754,-3.241320
1,images_train/984.jpg,-1.346954,-3.119461,-0.765971,-1.382550,-1.104675,-3.656271,-4.815436,-0.556942,-1.402286,...,0.011003,-3.968805,-2.694711,-4.196480,-2.880234,-1.210742,-1.605143,-4.859987,-0.837670,-0.967604
2,images_train/7127.jpg,-3.445498,-1.524573,-1.001654,-3.668335,-1.805517,-1.633496,-7.127826,-1.147802,-1.055816,...,-2.991777,-2.628053,-2.971074,-2.537039,-1.707429,1.013672,0.608460,-3.714998,-0.484735,0.138767
3,images_train/9609.jpg,1.114650,-2.167102,0.097881,-1.336255,0.853483,-0.374885,-2.369090,-2.273191,-1.143788,...,-1.248134,-0.633126,-1.723514,-2.638832,0.097149,4.647974,1.030138,-2.193836,1.044024,0.176043
4,images_train/5293.jpg,1.602650,-1.505817,3.029409,4.092412,1.711755,6.271253,4.173686,-2.177313,0.747789,...,-1.285806,-2.266481,-3.898053,2.295787,-1.749552,0.974188,1.258117,-1.975622,-1.278643,-1.941441


In [14]:
feats_train.shape,feats_test.shape

((10000, 2049), (2000, 2049))

In [15]:
pics_train = np.zeros((10000,3048))
for _,row in feats_train.iterrows():
    try:
        i = int(row[0].split('/')[1].split('.jpg')[0])
    except:
        i = int(row[0].split('/')[1].split('..jpg')[0])
    pics_train[i,:2048] = row.values[1:]
for _,row in feats_train_b.iterrows():
    try:
        i = int(row[0].split('/')[1].split('.jpg')[0])
    except:
        i = int(row[0].split('/')[1].split('..jpg')[0])
    pics_train[i,2048:] = row.values[1:]

In [16]:
pd.Series(pics_train.mean(0)).describe()

count    3048.000000
mean        0.286300
std         0.726504
min        -3.890733
25%         0.259784
50%         0.384236
75%         0.526699
max         4.441978
dtype: float64

In [17]:
pics_test = np.zeros((2000,3048))
for _,row in feats_test.iterrows():
    try:
        i = int(row[0].split('/')[1].split('.jpg')[0])
    except:
        i = int(row[0].split('/')[1].split('..jpg')[0])
    pics_test[i,:2048] = row.values[1:]
for _,row in feats_test_b.iterrows():
    try:
        i = int(row[0].split('/')[1].split('.jpg')[0])
    except:
        i = int(row[0].split('/')[1].split('..jpg')[0])
    pics_test[i,2048:] = row.values[1:]

In [18]:
pd.Series(pics_train.mean(0)).describe()

count    3048.000000
mean        0.286300
std         0.726504
min        -3.890733
25%         0.259784
50%         0.384236
75%         0.526699
max         4.441978
dtype: float64

In [19]:
tags_train = np.zeros((10000,100))
tags_test = np.zeros((2000,100))
tag_docs_train, tag_docs_test = [], []
for i in range(10000):
    with open('../tags_train/%d.txt' % (i,),'r') as f:
        tag_docs_train.append([word for line in f.read().split('\n') for word in line.split(':') if word])
for i in range(2000):
    with open('../tags_test/%d.txt' % (i,),'r') as f:
        tag_docs_test.append([word for line in f.read().split('\n') for word in line.split(':') if word])
tag_docs_train = preprocess(tag_docs_train,['lo','punct'])
tag_docs_test = preprocess(tag_docs_test,['lo','punct'])
for i in range(10000):
    tags_train[i] = model.infer_vector(tag_docs_train[i])
for i in range(2000):
    tags_test[i] = model.infer_vector(tag_docs_test[i])

In [20]:
pd.Series(np.abs(tags_test.mean(1))).describe()

count    2000.000000
mean        0.056461
std         0.058179
min         0.000003
25%         0.016499
50%         0.036197
75%         0.077830
max         0.396254
dtype: float64

In [21]:
pics_train = np.hstack([pics_train,tags_train])
pics_test = np.hstack([pics_test,tags_test])

In [23]:
pca = PCA(n_components=101)
pca.fit(pics_train)
pics_train = pca.transform(pics_train)
pics_test = pca.transform(pics_test)

In [3]:
def get_prediction(vecs,pics):
    dists = pairwise_distances(vecs,pics,metric='cosine')
    return dists.argsort(1)#[:,:20]

def get_top_20(descr_id):
    return preds[descr_id][:20]

def save_submission():
    data = []
    for i in range(2000):
        data.append(['%d.txt' % (i,),' '.join('%d.jpg' % (pic_id,) for pic_id in get_top_20(i))])
    pd.DataFrame(data,columns=['Descritpion_ID','Top_20_Image_IDs']).to_csv('submission.csv',index=False)
    
def map_20(preds):
    ranks = [np.argwhere(vec==i)[0][0] for i,vec in enumerate(preds)]
    return np.mean([(20-rank)/20 if rank<20 else 0 for rank in ranks])

In [18]:
rfr = RandomForestRegressor(n_estimators=500,max_depth=18,n_jobs=4)

In [19]:
text_train.shape,pics_train.shape

((10000, 101), (10000, 101))

In [20]:
# np.save('text_train',text_train)
# np.save('pics_train',pics_train)
# text_train = np.load('text_train.npy')
# pics_train = np.load('pics_train.npy')

In [21]:
## 500, 18
cv = KFold(n_splits=6,shuffle=True)
results = []
for train_index,test_index in cv.split(text_train,pics_train):
    rfr.fit(text_train[train_index],pics_train[train_index])
    raw_vectors = rfr.predict(text_train[test_index])
    preds = get_prediction(raw_vectors,pics_train[test_index])
    res = map_20(preds)
    ranks = [np.argwhere(vec==i)[0][0] for i,vec in enumerate(preds)]
    print(res,np.mean(ranks))
    results.append(res)
    

0.07909418116376725 335.99220155968806


KeyboardInterrupt: 

In [17]:
## 500, 15
cv = KFold(n_splits=6,shuffle=True)
results = []
for train_index,test_index in cv.split(text_train,pics_train):
    rfr.fit(text_train[train_index],pics_train[train_index])
    raw_vectors = rfr.predict(text_train[test_index])
    preds = get_prediction(raw_vectors,pics_train[test_index])
    res = map_20(preds)
    ranks = [np.argwhere(vec==i)[0][0] for i,vec in enumerate(preds)]
    print(res,np.mean(ranks))
    results.append(res)
    

0.07240551889622075 350.8188362327534
0.07633473305338932 338.7996400719856


KeyboardInterrupt: 

In [67]:
# test prediction
# preds = rfr.predict(text_test)
# preds = get_prediction(preds,pics_test)

In [83]:
# # train evaluation
# preds = rfr.predict(text_train)
# preds = get_prediction(preds,pics_train)
# print(np.mean([np.argwhere(vec==i)[0][0] for i,vec in enumerate(preds)]))
# print(np.median([np.argwhere(vec==i)[0][0] for i,vec in enumerate(preds)]))

0.201
0.0


In [71]:
save_submission()